### K-means clustering
The iris data is the most famous reference data in classification. It has three class, setosa, versicolor and virginica. 
Here, we classify the iris data using K-means clustering and check its performance.

First, import some basic libraries such as pandas, numpy and matplotlib. Next, import sklearn to implement K-means clustering and datasets.

In [ ]:
from sklearn import cluster, datasets
import matplotlib.pyplot as plt
iris = datasets.load_iris()

Iris is a dictionary. By the following code, we can check some information about iris data.
By printing iris, we can check that it has 150 arrays which have 4 features, length and width of sepal and petal. 

In [ ]:
print(iris)
print(len(iris.data))
print(iris.target_names)
print(iris.target)

The Seaborn library offers a visualized iris data using pairs of features.

In [ ]:
import seaborn as sns

sns.set(style = 'ticks', color_codes = True)
iris1 = sns.load_dataset("iris")
g = sns.pairplot(iris1, hue = "species", palette = "husl")

Use the sepal width and sepal length only for classification. Using cluster module in sklearn, we can easily run a K-means clustering algorithm.  

In [ ]:
X = iris.data[:,:2]
y_iris = iris.target

km2 = cluster.KMeans(n_clusters = 2).fit(X)
km3 = cluster.KMeans(n_clusters = 3).fit(X)
km4 = cluster.KMeans(n_clusters = 4).fit(X)

plt.figure(figsize=(9, 3))
plt.subplot(131)
plt.scatter(X[:, 0], X[:, 1], c=km2.labels_)
plt.title("K=2, J=%.2f" % km2.inertia_)
plt.subplot(132)
plt.scatter(X[:, 0], X[:, 1], c=km3.labels_)
plt.title("K=3, J=%.2f" % km3.inertia_)
plt.subplot(133)
plt.scatter(X[:, 0], X[:, 1], c=km4.labels_)
plt.title("K=4, J=%.2f" % km4.inertia_)


To check its performance, it is convinent to use the following function which gives an integer vector of the samples' labels.

In [ ]:
def kmeans(X,K):
    km = cluster.KMeans(n_clusters = K).fit(X)
    return km.labels_

In [ ]:
print(kmeans(X,3))
print(iris.target)

As we already know that the iris data are classified by 3 classes, assume $K=3$.  

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.scatter(X[:, 0], X[:, 1], c=km3.labels_)
plt.title("K=3, J=%.3f, using K-means clustering" % km3.inertia_)
plt.subplot(122)
plt.scatter(X[:, 0], X[:, 1], c = y_iris)
plt.title("True labels")

Inertia of K-means clustering is based on Euclidean distance between samples and centroids $\{\mu_k\}$.  

In the sense of Euclidean distance, K-means clustering works well. However, two classes are not classified by the distance of sample's sepal width and sepal length.

So, we can say that sepal width and sepal length are not good features for K-means clustering.

Instead of sepal, if we use the width and length of petal...

In [ ]:
XX = iris.data[:,2:4]

km = cluster.KMeans(n_clusters = 3).fit(XX)

plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.scatter(XX[:, 0], XX[:, 1], c=km.labels_)
plt.title("K=3, J=%.3f, using K-means clustering" % km.inertia_)
plt.subplot(122)
plt.scatter(XX[:, 0], XX[:, 1], c = y_iris)
plt.title("True labels")

## Gaussian mixture model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=400, centers=4, cluster_std=0.60, random_state=0)
X = X[:, ::-1]

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=40);

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(4, random_state=0)
labels = kmeans.fit(X).predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

In [ ]:
from scipy.spatial.distance import cdist

def plot_kmeans(kmeans, X, n_clusters=4, rseed=0, ax=None):
    labels = kmeans.fit_predict(X)

    # plot the input data
    ax = ax or plt.gca()
    ax.axis('equal')
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)

    # plot the representation of the KMeans model
    centers = kmeans.cluster_centers_
    radii = [cdist(X[labels == i], [center]).max()
             for i, center in enumerate(centers)]
    for c, r in zip(centers, radii):
        ax.add_patch(plt.Circle(c, r, fc='#CCCCCC', lw=3, alpha=0.5, zorder=1))

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X)

In [ ]:
rng = np.random.RandomState(13)
X_stretched = np.dot(X, rng.randn(2, 2))

kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X_stretched)

Result for the transformed data.

In [ ]:
from sklearn.mixture import GaussianMixture as GMM
gmm = GMM(n_components=4).fit(X)
labels = gmm.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis')

In [ ]:
probs = gmm.predict_proba(X)
print(probs[:5].round(3))

The first five elements which measures the probability that any point belongs to the given cluster.

In [ ]:
size = 50 * probs.max(1) ** 2  
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=size);

Results of GMM reflecting clustering probabilities.

In [ ]:
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [ ]:
gmm = GMM(n_components=4, random_state=42)
plot_gmm(gmm, X)

In [ ]:
gmm = GMM(n_components=4, covariance_type='full', random_state=42)
plot_gmm(gmm, X_stretched)

### GMM as Density Estimation

In [ ]:
from sklearn.datasets import make_moons
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
plt.scatter(Xmoon[:, 0], Xmoon[:, 1]);

In [ ]:
gmm2 = GMM(n_components=2, covariance_type='full', random_state=0)
plot_gmm(gmm2, Xmoon)

In [ ]:
Xnew = gmm2.sample(400)
plt.scatter(Xnew[0][:, 0], Xnew[0][:, 1]);

In [ ]:
gmm16 = GMM(n_components=16, covariance_type='full', random_state=0)
plot_gmm(gmm16, Xmoon, label=False)

In [ ]:
Xnew = gmm16.sample(400)
plt.scatter(Xnew[0][:, 0], Xnew[0][:, 1]);

Generated data from trained GMM.

In [ ]:
n_components = np.arange(1, 21)
models = [GMM(n, covariance_type='full', random_state=0).fit(Xmoon)
          for n in n_components]

plt.plot(n_components, [m.bic(Xmoon) for m in models], label='BIC')
plt.plot(n_components, [m.aic(Xmoon) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [ ]:
gmm9 = GMM(n_components=9, covariance_type='full', random_state=0)
plot_gmm(gmm9, Xmoon, label=False)

In [ ]:
Xnew = gmm9.sample(400)
plt.scatter(Xnew[0][:, 0], Xnew[0][:, 1]);

#### References
https://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html